# NYC Highschool´s SAT Analysis

## Reading the Datasets

In this project, we are going to have a look at the different factors that may affect the SAT performance for high school students across New York City. 

The main purpose it´s to practise the data-wrangling skills to be able to build a robust dataset out of different sources. 

In [1]:
import pandas as pd

In [108]:
# We are going to import all the CSV files that contain each of the datasets to be used later on
# Then we are going go  add the previous datasets to a dictionary to make the accessment more efficient and organized

datasets = ["ap_2010", "class_size", "demographics", "graduation", "hs_directory", "sat_results"]
data = {}

for element in datasets:
    data[element] = pd.read_csv("data/" + element + ".csv")

Let´s read also the information about the surveys. It is divided in two .txt files, so we need to concatenate them.

In [16]:
survey_all = pd.read_csv("data/survey_all.txt", delimiter="\t", encoding="windows-1252")
survey_d75 = pd.read_csv("data/survey_d75.txt", delimiter="\t", encoding="windows-1252")
survey = pd.concat([survey_all, survey_d75], axis = 0, sort = False)
survey.head(3)


,dbn,bn,schoolname,d75,studentssurveyed,highschool,schooltype,rr_s,rr_t,rr_p,...,s_N_q14e_3,s_N_q14e_4,s_N_q14f_1,s_N_q14f_2,s_N_q14f_3,s_N_q14f_4,s_N_q14g_1,s_N_q14g_2,s_N_q14g_3,s_N_q14g_4
0,75K004,K004,P.S. K004,1,Yes,0.0,District 75 Special Education,38.0,90,72,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,75K036,K036,P.S. 36,1,Yes,NaN,District 75 Special Education,70.0,69,44,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,75K053,K053,P.S. K053,1,Yes,NaN,District 75 Special Education,94.0,97,53,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


There are more than 2700 columns, nearly all of which we don´t need. We are going to filter to keep only the relevant ones. 

In [17]:
relevant_columns = ["dbn", "rr_s", "rr_t", "rr_p", "N_s", "N_t", "N_p", "saf_p_11", "com_p_11", "eng_p_11", "aca_p_11", 
                     "saf_t_11", "com_t_11", "eng_t_11", "aca_t_11", "saf_s_11", "com_s_11", "eng_s_11", "aca_s_11",
                     "saf_tot_11", "com_tot_11", "eng_tot_11", "aca_tot_11"]

survey = survey[relevant_columns]  #The new dataset will only have the relevant columns  ¡
survey.rename(index = str, columns = {"dbn": "DBN"}, inplace = True) #Renaming the DBN column to normalize it with the others datasets

data["survey"] = survey     # We can now include the survery dataset in the dictionary
data["survey"].head(3)


,DBN,rr_s,rr_t,rr_p,N_s,N_t,N_p,saf_p_11,com_p_11,eng_p_11,...,eng_t_11,aca_t_11,saf_s_11,com_s_11,eng_s_11,aca_s_11,saf_tot_11,com_tot_11,eng_tot_11,aca_tot_11
0,75K004,38.0,90,72,8.0,81.0,244.0,9.1,8.6,8.4,...,6.4,6.4,7.6,5.8,7.6,6.3,8.0,7.0,7.5,7.1
1,75K036,70.0,69,44,97.0,43.0,115.0,9.2,8.7,8.6,...,6.1,7.2,6.6,6.9,7.3,7.7,7.5,7.6,7.3,7.9
2,75K053,94.0,97,53,131.0,75.0,189.0,8.8,8.1,8.0,...,6.9,8.0,8.2,7.7,8.6,8.5,8.2,7.8,7.8,8.2


What we're mainly interested in is the *sat_results* dataset. This data set contains the SAT scores for each high school in New York City. We eventually want to correlate selected information from this data set with information in the other datasets. Let´s explore it.

In [5]:
data["sat_results"].head(3)

,DBN,SCHOOL NAME,Num of SAT Test Takers,SAT Critical Reading Avg. Score,SAT Math Avg. Score,SAT Writing Avg. Score
0,01M292,HENRY STREET SCHOOL FOR INTERNATIONAL STUDIES,29,355,404,363
1,01M448,UNIVERSITY NEIGHBORHOOD HIGH SCHOOL,91,383,423,366
2,01M450,EAST SIDE COMMUNITY SCHOOL,70,377,402,370


In this dataset, we can observe that the first column *DBN* appears to be a unique identifier. We can see as well that the last three columns are the result of the SAT in each of the different parts of it. For the purpose of this project, we will add them all together in a new column. 

In [24]:
cols = ['SAT Math Avg. Score', 'SAT Critical Reading Avg. Score', 'SAT Writing Avg. Score']
data["sat_results"]["sat_score"] = 0

#for the columns in col, we are going to transform them to numeric (errors = NAN) and add them to the *sat_score* column

for c in cols:
    data["sat_results"]["sat_score"] += pd.to_numeric(data["sat_results"][c], errors="coerce")
    
data["sat_results"].head(3)  

,DBN,SCHOOL NAME,Num of SAT Test Takers,SAT Critical Reading Avg. Score,SAT Math Avg. Score,SAT Writing Avg. Score,sat_score
0,01M292,HENRY STREET SCHOOL FOR INTERNATIONAL STUDIES,29,355.0,404.0,363.0,1122.0
1,01M448,UNIVERSITY NEIGHBORHOOD HIGH SCHOOL,91,383.0,423.0,366.0,1172.0
2,01M450,EAST SIDE COMMUNITY SCHOOL,70,377.0,402.0,370.0,1149.0


Now let´s check the other datasets.

In [6]:
data["ap_2010"].head(3)

,DBN,SchoolName,AP Test Takers,Total Exams Taken,Number of Exams with scores 3 4 or 5
0,01M448,UNIVERSITY NEIGHBORHOOD H.S.,39.0,49.0,10.0
1,01M450,EAST SIDE COMMUNITY HS,19.0,21.0,NaN
2,01M515,LOWER EASTSIDE PREP,24.0,26.0,24.0


In [25]:
data["demographics"].head(3)

,DBN,Name,schoolyear,fl_percent,frl_percent,total_enrollment,prek,k,grade1,grade2,...,black_num,black_per,hispanic_num,hispanic_per,white_num,white_per,male_num,male_per,female_num,female_per
0,01M015,P.S. 015 ROBERTO CLEMENTE,20052006,89.4,NaN,281,15,36,40,33,...,74,26.3,189,67.3,5,1.8,158.0,56.2,123.0,43.8
1,01M015,P.S. 015 ROBERTO CLEMENTE,20062007,89.4,NaN,243,15,29,39,38,...,68,28.0,153,63.0,4,1.6,140.0,57.6,103.0,42.4
2,01M015,P.S. 015 ROBERTO CLEMENTE,20072008,89.4,NaN,261,18,43,39,36,...,77,29.5,157,60.2,7,2.7,143.0,54.8,118.0,45.2


In [26]:
data["graduation"].head(3)

,Demographic,DBN,School Name,Cohort,Total Cohort,Total Grads - n,Total Grads - % of cohort,Total Regents - n,Total Regents - % of cohort,Total Regents - % of grads,...,Regents w/o Advanced - n,Regents w/o Advanced - % of cohort,Regents w/o Advanced - % of grads,Local - n,Local - % of cohort,Local - % of grads,Still Enrolled - n,Still Enrolled - % of cohort,Dropped Out - n,Dropped Out - % of cohort
0,Total Cohort,01M292,HENRY STREET SCHOOL FOR INTERNATIONAL,2003,5,s,NaN,s,NaN,NaN,...,s,NaN,NaN,s,NaN,NaN,s,NaN,s,NaN
1,Total Cohort,01M292,HENRY STREET SCHOOL FOR INTERNATIONAL,2004,55,37,67.3,17,30.9,45.9,...,17,30.9,45.9,20,36.4,54.1,15,27.3,3,5.5
2,Total Cohort,01M292,HENRY STREET SCHOOL FOR INTERNATIONAL,2005,64,43,67.2,27,42.2,62.8,...,27,42.2,62.8,16,25.0,37.2,9,14.1,9,14.1


In [27]:
data["hs_directory"].rename(index=str, columns={"dbn": "DBN"}, inplace = True)      #Changing the DBN column to match the others datasets 
data["hs_directory"].head(3)

,DBN,school_name,borough,building_code,phone_number,fax_number,grade_span_min,grade_span_max,expgrade_span_min,expgrade_span_max,...,priority08,priority09,priority10,Location 1,Community Board,Council District,Census Tract,BIN,BBL,NTA
0,17K548,Brooklyn School for Music & Theatre,Brooklyn,K440,718-230-6250,718-230-6262,9.0,12,NaN,NaN,...,NaN,NaN,NaN,"883 Classon Avenue\nBrooklyn, NY 11225\n(40.67...",9.0,35.0,213.0,3029686.0,3.011870e+09,Crown Heights South ...
1,09X543,High School for Violin and Dance,Bronx,X400,718-842-0687,718-589-9849,9.0,12,NaN,NaN,...,NaN,NaN,NaN,"1110 Boston Road\nBronx, NY 10456\n(40.8276026...",3.0,16.0,135.0,2004526.0,2.026340e+09,Morrisania-Melrose ...
2,09X327,Comprehensive Model School Project M.S. 327,Bronx,X240,718-294-8111,718-294-8109,6.0,12,NaN,NaN,...,NaN,NaN,NaN,"1501 Jerome Avenue\nBronx, NY 10452\n(40.84241...",4.0,14.0,209.0,2008336.0,2.028590e+09,West Concourse ...


In [28]:
data["class_size"].head(3)

,CSD,BOROUGH,SCHOOL CODE,SCHOOL NAME,GRADE,PROGRAM TYPE,CORE SUBJECT (MS CORE and 9-12 ONLY),CORE COURSE (MS CORE and 9-12 ONLY),SERVICE CATEGORY(K-9* ONLY),NUMBER OF STUDENTS / SEATS FILLED,NUMBER OF SECTIONS,AVERAGE CLASS SIZE,SIZE OF SMALLEST CLASS,SIZE OF LARGEST CLASS,DATA SOURCE,SCHOOLWIDE PUPIL-TEACHER RATIO,DBN
0,01,M,M015,P.S. 015 Roberto Clemente,0K,GEN ED,-,-,-,19.0,1.0,19.0,19.0,19.0,ATS,NaN,01M015
1,01,M,M015,P.S. 015 Roberto Clemente,0K,CTT,-,-,-,21.0,1.0,21.0,21.0,21.0,ATS,NaN,01M015
2,01,M,M015,P.S. 015 Roberto Clemente,01,GEN ED,-,-,-,17.0,1.0,17.0,17.0,17.0,ATS,NaN,01M015


The *class_size* dataset does not have the *DBN* column. However, it can be obtained by the combination of the *CSD* and *School Code* columns.

The *CSD* column need a transformation to be formed of two characters even if it´s an only one digit number. Therefore, we have applied the following function to the *CSD* column. 

In [29]:
def padded_csd(csd):
    csd = str(csd)
    if len(csd) == 1:
        csd = "0" + csd
    return csd

data["class_size"]["CSD"] = data["class_size"]["CSD"].apply(padded_csd)

#Now we can combine the previously metions columns to create the *CSD* field

data["class_size"]["DBN"] =  data["class_size"]["CSD"] + data["class_size"]["SCHOOL CODE"] 
data["class_size"].head(3)

,CSD,BOROUGH,SCHOOL CODE,SCHOOL NAME,GRADE,PROGRAM TYPE,CORE SUBJECT (MS CORE and 9-12 ONLY),CORE COURSE (MS CORE and 9-12 ONLY),SERVICE CATEGORY(K-9* ONLY),NUMBER OF STUDENTS / SEATS FILLED,NUMBER OF SECTIONS,AVERAGE CLASS SIZE,SIZE OF SMALLEST CLASS,SIZE OF LARGEST CLASS,DATA SOURCE,SCHOOLWIDE PUPIL-TEACHER RATIO,DBN
0,01,M,M015,P.S. 015 Roberto Clemente,0K,GEN ED,-,-,-,19.0,1.0,19.0,19.0,19.0,ATS,NaN,01M015
1,01,M,M015,P.S. 015 Roberto Clemente,0K,CTT,-,-,-,21.0,1.0,21.0,21.0,21.0,ATS,NaN,01M015
2,01,M,M015,P.S. 015 Roberto Clemente,01,GEN ED,-,-,-,17.0,1.0,17.0,17.0,17.0,ATS,NaN,01M015


In the *hs_directory* dataset we find the "Location 1" field that contains the latitude and longitude coodinates for each school. Let´s see an example of the column to check how we can extract the information about its coordiantes

In [40]:
data["hs_directory"]["Location 1"][0]

'883 Classon Avenue\nBrooklyn, NY 11225\n(40.67029890700047, -73.96164787599963)'

To information we want is inside the parentheses. We are going to create a function with a regular expression to extract it. 

In [109]:
import re # Import the RegEx library 

def regex_lat(string):
    coor = re.findall("\(.+\)",string)[0]
    coor = coor.replace("(","").replace(")","")
    lat = coor.split(sep=", ")[0]
    return lat

def regex_lon(string):
    coor = re.findall("\(.+\)",string)[0]
    coor = coor.replace("(","").replace(")","")
    lon = coor.split(sep=", ")[1]
    return lon

data["hs_directory"]["lat"] = data["hs_directory"]["Location 1"].apply(regex_lat)
data["hs_directory"]["long"] = data["hs_directory"]["Location 1"].apply(regex_lon)
data["hs_directory"].head(3)

,dbn,school_name,borough,building_code,phone_number,fax_number,grade_span_min,grade_span_max,expgrade_span_min,expgrade_span_max,...,priority10,Location 1,Community Board,Council District,Census Tract,BIN,BBL,NTA,lat,long
0,17K548,Brooklyn School for Music & Theatre,Brooklyn,K440,718-230-6250,718-230-6262,9.0,12,NaN,NaN,...,NaN,"883 Classon Avenue\nBrooklyn, NY 11225\n(40.67...",9.0,35.0,213.0,3029686.0,3.011870e+09,Crown Heights South ...,40.67029890700047,-73.96164787599963
1,09X543,High School for Violin and Dance,Bronx,X400,718-842-0687,718-589-9849,9.0,12,NaN,NaN,...,NaN,"1110 Boston Road\nBronx, NY 10456\n(40.8276026...",3.0,16.0,135.0,2004526.0,2.026340e+09,Morrisania-Melrose ...,40.8276026690005,-73.90447525699966
2,09X327,Comprehensive Model School Project M.S. 327,Bronx,X240,718-294-8111,718-294-8109,6.0,12,NaN,NaN,...,NaN,"1501 Jerome Avenue\nBronx, NY 10452\n(40.84241...",4.0,14.0,209.0,2008336.0,2.028590e+09,West Concourse ...,40.842414068000494,-73.91616158599965
